In [ ]:
# pip install pandas numpy matplotlib statsmodels scikit-learn
import pandas as pd, numpy as np, matplotlib.pyplot as plt, math
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

df = pd.read_csv("dados_onibus.csv")

# cria ordem de leitura
df["row_id"] = np.arange(len(df))
weekday_order = ["Seg","Ter","Qua","Qui","Sex","Sab","Dom"]
weekday_to_idx = {d:i for i,d in enumerate(weekday_order)}
df["weekday_idx"] = df["dia"].map(weekday_to_idx)

# ordena por linha e ordem de aparecimento
df = df.sort_values(["linha","row_id"]).reset_index(drop=True)

# contador sequencial de dias para cada linha, o eixo temporal do modelo.
df["t_seq"] = df.groupby("linha").cumcount()

capacidade_onibus = 50
resultados, previsoes = {}, []

for linha, g in df.groupby("linha"):
    # Para cada linha de ônibus, cria uma série temporal s indexada por t_seq
    s = g.set_index("t_seq")["requests"]

    # SARIMAX semanal 
    model = SARIMAX(s, order=(1,0,1), seasonal_order=(1,1,0,7),
                    enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)

    # previsão 1 passo à frente
    fc = res.get_forecast(steps=1)
    mean = float(fc.predicted_mean.iloc[0])
    lo, hi = fc.conf_int().iloc[0].tolist() #limites inferior e superior do intervalo de confiança

    # erro de treino
    fitted = res.fittedvalues.dropna()
    y_true = s.loc[fitted.index]
    mse = mean_squared_error(y_true, fitted) #Compara com os valores reais da série e mede o erro quadrático médio (MSE)

    prev_round = int(round(mean))
    buses = math.ceil(max(prev_round,0) / capacidade_onibus)

    resultados[linha] = dict(prev=prev_round, buses=buses, mse=mse, lo=lo, hi=hi, last=s.iloc[-1])
    previsoes.append({"linha": linha, "next_day": int(s.index[-1])+1, "requests": prev_round})

# DataFrame final
df_prev = pd.DataFrame(previsoes).sort_values("linha")

# Resumo por linha
for linha, r in resultados.items():
    print(f"Linha {linha}: prev={r['prev']} ônibus={r['buses']}  MSE={r['mse']:.2f}  "
          f"(Intervalo≈[{r['lo']:.1f},{r['hi']:.1f}])")


Linha 1: prev=958 ônibus=20  MSE=8014985.97  (Intervalo≈[668.7,1248.1])
Linha 2: prev=2272 ônibus=46  MSE=797403.26  (Intervalo≈[1770.5,2773.9])
Linha 3: prev=693 ônibus=14  MSE=690993.18  (Intervalo≈[396.9,988.5])
Linha 4: prev=770 ônibus=16  MSE=660616.89  (Intervalo≈[535.1,1005.2])
Linha 5: prev=213 ônibus=5  MSE=142240.17  (Intervalo≈[-100.7,526.1])


c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\leona\AppData\Local